In [ ]:
!pip install -U datasets

In [ ]:
!wget https://object.pouta.csc.fi/OPUS-NLLB/v1/moses/ru-tt.txt.zip -O NLLB.zip
!unzip NLLB.zip -d NLLB

!wget https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2024/moses/ru-tt.txt.zip -O OpenSubtitles.zip
!unzip OpenSubtitles.zip -d OpenSubtitles

!wget https://object.pouta.csc.fi/OPUS-wikimedia/v20230407/moses/ru-tt.txt.zip -O wikimedia.zip
!unzip wikimedia.zip -d wikimedia

!wget https://object.pouta.csc.fi/OPUS-Tanzil/v1/moses/ru-tt.txt.zip -O Tanzil.zip
!unzip Tanzil.zip -d Tanzil

!wget https://object.pouta.csc.fi/OPUS-Tatoeba/v2023-04-12/moses/ru-tt.txt.zip -O Tatoeba.zip
!unzip Tatoeba.zip -d Tatoeba


In [ ]:
from datasets import load_dataset
import pandas as pd

datasets = []


datasets.append(
    load_dataset('AigizK/tatar-russian-parallel-corpora', split="train").to_pandas().rename(columns={"rus": "ru"})
)

In [ ]:
def extract_data(file):
  with open(file + '.ru') as f:
    dataset_ru = [line.rstrip() for line in f]
  with open(file + '.tt') as f:
    dataset_tat = [line.rstrip() for line in f]
  return pd.DataFrame({'tat': dataset_tat, 'ru': dataset_ru}, columns=['tat', 'ru'])

In [ ]:
files = ['OpenSubtitles/OpenSubtitles.ru-tt', 'wikimedia/wikimedia.ru-tt', 'Tanzil/Tanzil.ru-tt', 'Tatoeba/Tatoeba.ru-tt']

for file in files:
  datasets.append(extract_data(file))

In [ ]:
import re

def clean_text(text):
    if not isinstance(text, str):
        return ""

    # Удаляет всё между (), включая скобки
    text = re.sub(r'\([^)]*\)', '', text)
    # Удаляет всё между [], включая скобки
    text = re.sub(r'\[[^]]*\]', '', text)

    text = re.sub(r'<.*?>', ' ', text)                      # HTML
    text = re.sub(r'\S+@\S+', ' ', text)                    # Emails
    text = re.sub(r'http\S+|www\.\S+', ' ', text)           # URLs
    text = re.sub(r'[\U00010000-\U0010ffff]', '', text)     # Emojis
    text = re.sub(r'[^\s.!?,0-9а-яА-ЯёЁүҮөӨңҢҗҖһҺәӘ-]', ' ', text)  # спецсимволы
    text = re.sub(r'\s+', ' ', text)                        # множественные пробелы
    return text.strip()

def normalize_text(text):
    text = text.lower()                     # нижний регистр
    text = re.sub(r'\s+', ' ', text)        # множественные пробелы
    return text.strip()

In [ ]:
for i in range(len(datasets)):
    datasets[i] = datasets[i][~datasets[i]['ru'].str.contains(r'[A-Za-z]', na=False)]
    datasets[i] = datasets[i][~datasets[i]['tat'].str.contains(r'[A-Za-z]', na=False)]
    datasets[i]['ru'] = datasets[i]['ru'].astype(str).apply(clean_text).apply(normalize_text)
    datasets[i]['tat'] = datasets[i]['tat'].astype(str).apply(clean_text).apply(normalize_text)
    datasets[i].dropna(inplace=True)
    datasets[i].drop_duplicates(inplace=True)
    datasets[i] = datasets[i][(datasets[i]['ru'].str.strip() != '') & (datasets[i]['tat'].str.strip() != '')]
    datasets[i] = datasets[i][
      (datasets[i]["ru"].str.split().str.len().between(1, 25)) &
      (datasets[i]["tat"].str.split().str.len().between(1, 25))
    ]


LETTERS_SET = set("абвгдеёжзийклмнопрстуфхцчшщъыьэюяәөүҗңһҺӘӨҮҢҖАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ")

def fast_check(text):
    if not isinstance(text, str):
        return False
    count = 0
    for char in text:
        if char in LETTERS_SET:
            count += 1
            if count >= 2:
                return True
    return False

In [ ]:
corpus_par = pd.concat(datasets)

In [ ]:
corpus_par = corpus_par[
    corpus_par['tat'].apply(fast_check) &
    corpus_par['ru'].apply(fast_check)
]

In [ ]:
corpus_par

,tat,ru
0,рэп-баттл ярдәмендә фигыльләрне өйрәнәбез .,учим глаголы при помощи рэп-баттла .
1,россия өчен укытучы проектында катнашучылар ав...,участники проекта учитель для россии о сельски...
2,2015 елда берничә төбәктә россия өчен укытучы ...,в 2015 году в нескольких регионах стартовала п...
3,нәкъ шундый ук проектлар күп кенә илләрдә һинд...,точно такие же проекты существуют во многих ст...
4,болар барысы да шул исәптән мәктәпкә мөнәсәбәт...,"все это придумано в том числе и для того, чтоб..."
...,...,...
3296,бу минем өчен җитәрлек түгел.,для меня этого недостаточно.
3297,ул үз бүлмәсендә.,он в своей комнате.
3298,ул үз бүлмәсендә.,он у себя в комнате.
3299,алга һәм җыр белән!,вперёд и с песней!
